## Self Replicating Notebook

#### Connect to current ArcGIS Online portal and import required modules

In [ ]:
from arcgis.gis import GIS
gis = GIS("home")

In [ ]:
import datetime
import requests

#### Settings

In [ ]:
snapshotUrl = "https://notebooksservices9.arcgis.com/admin/notebooks/snapshots"
itemId = "497bfdc9754f4b7cb371e28a7449aedb"
today = datetime.datetime.now().strftime("%Y-%m-%d")

### Create Notebook Snapshot

In [ ]:
snapshotName = f"snapshot_{today}"
snapshotName

In [ ]:
createSnapshotUrl = f"{snapshotUrl}/create?f=json&token={gis._con.token}"
createSnapshotParams = {
    "itemId": itemId,
    "name": snapshotName
}

In [ ]:
createSnapshotResponse = requests.post(createSnapshotUrl, createSnapshotParams).json()
if createSnapshotResponse["status"] == "success":
    snapshotResourceKey = createSnapshotResponse["snapshotResourceKey"]

### Convert Notebook Snapshot to Item

In [ ]:
notebookTitle = f"Backup Notebook ({today})"
notebookTitle

In [ ]:
convertToItemUrl = f"{snapshotUrl}/convertToItem?f=json&token={gis._con.token}"
convertToItemParams = {
    "itemId": itemId,
    "resourceKey": snapshotResourceKey,
    "notebookTitle": notebookTitle
}

In [ ]:
convertToItemResponse = requests.post(convertToItemUrl, convertToItemParams).json()
if convertToItemResponse["status"] == "success":
    print(f"Created Notebook with itemId: {convertToItemResponse['itemId']}")

In [ ]:
notebookItem = gis.content.get(convertToItemResponse['itemId'])
notebookItem

### Execute Notebook

In [ ]:
executeNotebookUrl = f"https://notebooksservices9.arcgis.com/admin/notebooks/executeNotebook?f=json&token={gis._con.token}"
executeNotebookParams = {
    "itemId": convertToItemResponse['itemId']
}

In [ ]:
executeNotebookResponse = requests.post(executeNotebookUrl, executeNotebookParams).json()

In [ ]:
resp = requests.get(f"{executeNotebookResponse['jobUrl']}?f=json&token={gis._con.token}")
resp.json()